In [ ]:
## loading dependent packages
import numpy as np
import pandas as pd
from typing import List, Tuple, Union
from numpy.typing import NDArray

In [ ]:
## the state vector (x_k) at time step k will contain position and velocity components:
## X = [x, y, v_x, v_y]^T
DT: float = 0.1 ## time step is 10 frames per second

In [ ]:
class ConstantVelocityKalmanFilter:
    """
    Implements a linear Kalman Filter using a Constant Velocity (CV) model
    for player tracking.

    The state estimation process involves two phases:
    1. Filtering: Applying predict/update steps on the input data for smoothing.
    2. Prediction: Applying only the predict the predict step for future frames.
    """

    ## Class Attributes
    dt: float
    A: NDArray[np.float64]  ## state transition matrix (4x4)
    H: NDArray[np.float64]  ## measurement matrix (4x4)
    I: NDArray[np.float64]  ## identity matrix (4x4)
    Q: NDArray[np.float64]  ## process noise covariance (4x4)
    R: NDArray[np.float64]  ## measurement noise covariance (2x2)
    X: Union[NDArray[np.float64], None]  ## state vector [x, y, v_x, v_y]^T (4x1)
    P: Union[NDArray[np.float64], None]  ## covariance matrix (4x4)
    """
    def __init__(self, process_noise_std: float = 0.5, measurement_noise_std: float = 0.1) -> None:
        self.dt = DT

        ## --- Model Matrices (A, H) ---

        ## state transition matrix (A, 4x4) - Constant Velocity Model
        self.A = np.array([
            [1, 0, self.dt, 0],
            [0, 1, 0, self.dt],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])

        ## measurement matrix (H, 2x4): We only observe (x, y)
        self.H = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])

        ## identity matrix for the update step
        self.I = np.identity(self.A.shape[0])

        ## --- Noise Covariance Matrices (Q, R) ---
        ## process noise covariance (Q, 4x4)
        ## Q is often tuned by modeling the white noise acceleration (q)
        q = process_noise_std**2
        self.Q = q * np.array([
            [self.dt**4/4, 0, self.dt**3/2, 0],
            [0, self.dt**4/4, 0, self.dt**3/2],
            [self.dt**3/2, 0, self.dt**2, 0],
            [0, self.dt**3/2, 0, self.dt**2]
        ])

        ## measurement noise covariance (R, 2x2)
        ## R is the uncertainty in the measurement readings (x, y)
        r = measurement_noise_std**2
        self.R = r*np.identity(2)

        ## state and covariance initialized to None
        self.X = None ## state vector [x, y, v_x, v_y]^T
        self.Y = None ## covariance matrix (4x4)

    def initialize(self, initial_state: np.ndarray, initial_covariance: np.ndarray) -> None:
        """Sets the initial state and covariance.""" 
        self.X = initial_state
        self.Y = initial_covariance

    def predict(self) -> Tuple[np.npdarray, np.ndarray]:
        """Performs the time update (prediction step)."""
        ## project the state ahead of X_k = A * X_{k-1}
        X_predicted = self.A @ self.X

        ## project the error covariance ahead of P_k = A * P_{k-1} * A^T + Q
        P_predicted = self.A @ self.P @ self.A.T + self.Q

        ## store the predictions
        self.X = X_predicted
        self.P = P_predicted

        return X_predicted, P_predicted
    
    def update(self, measurement: np.ndarray) -> np.ndarray:
        """
        Performs the measurement update (correction step). Uses the latest prediction
        (self.X, self.P) and the new measurement (Z).

        Arguments:
            measurement: the observed [x, y] position (2x1).
        """
        ## compute the Kalman Gain (K): K = P_pred * H^T * (H * P_pred * H^T + R)^{-1}
        S = self.H @ self.P @ self.H.T + self.R  ## innovation covariance
        K = self.P @ self.H.T @ np.linalg.inv(s)  ## Kalman Gain

        ## updating the state estimate: X _new = X_pred + K * Z - H * X_pred)
        Y = measurement - (self.H @ self.X)  ## innovation (residual)
        self.X = self.X + (K @ Y)

        ## updating the error covariance: P_new = (I - K * H) * P_pred
        self.P(self.I - K @ self.H) @ self.P

        return self.X